In [1]:
import os
os.chdir("D:\pp\Price-Analysis-DataScience")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    target_column: str


In [3]:
from src.price_prediction.constants import *
from src.price_prediction.utils.common import read_yaml, create_directories


In [37]:
class Configuration:
    def __init__(self, 
    config_filepath = CONFIG_FILE_PATH,
    params_filepath = PARAMS_FILE_PATH,
    schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self):
        config = self.config.model_trainer
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])
        return ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = str(list(schema.keys())[0])
        )
    
    

In [38]:
import pandas as pd
import joblib
import os
from price_prediction import logger
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import GradientBoostingRegressor

In [39]:
class ModelTraining:
    def __init__(self, config):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        X_train = train_data.drop(columns = [self.config.target_column])
        y_train = train_data[self.config.target_column]
        X_test = test_data.drop(columns = [self.config.target_column])
        y_test = test_data[self.config.target_column]
        numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
        categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
        preprocessor = ColumnTransformer(
                transformers=[
                    ('num', StandardScaler(), numerical_features),
                    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
                ])
        model = Pipeline(
            steps = [
                ('preprocessor', preprocessor),
                ('regressor', GradientBoostingRegressor(random_state=42))
            ]
        )

        model.fit(X_train, y_train)
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))

In [40]:
try:
    config = Configuration()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTraining(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-04-11 17:41:43,928: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-11 17:41:43,932: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-11 17:41:43,936: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-11 17:41:43,938: INFO: common: created directory at: artificats]
[2024-04-11 17:41:43,939: INFO: common: created directory at: artifacts/model_training]
sellingprice
